In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
from catalyst.utils import get_device, set_global_seed, prepare_cudnn
from catalyst.dl import SupervisedRunner, ConfusionMatrixCallback
from src.nn import BACTERIA, CrossEntropyLoss, get_classification_model, get_class_names
prepare_cudnn(deterministic=True)
set_global_seed(0)
class_names = get_class_names()
num_classes = len(class_names)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
torch.cuda.empty_cache()
model = get_classification_model(num_classes)
model.train()
criterion = CrossEntropyLoss()
callbacks = [ConfusionMatrixCallback(num_classes=num_classes, class_names=class_names)]
dataset = BACTERIA(target='label')
loaders = next(dataset.crossval(kfold=4, batch_size=9))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
runner = SupervisedRunner(device=get_device())
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    num_epochs=100,
    logdir='./logs/classification',
    # verbose=True,
)

In [ ]:
! zip -r logs/classification.zip logs/classification
! git add src logs
! git commit -m 'Changed from Colab'
! git push -u origin master